In [1]:
# !pip install kivy pillow
# !pip install opencv-python numpy matplotlib
# !pip install ultralytics
# !pip install --upgrade pip setuptools
# !pip install buildozer
# !pip install --upgrade buildozer cython



SyntaxError: invalid syntax (2370593526.py, line 7)

In [ ]:
# !pip uninstall opencv-python
# !pip install opencv-python==4.5.1


In [13]:
import urllib.request

# Download YOLOv3 weights
urllib.request.urlretrieve(
    "https://pjreddie.com/media/files/yolov3.weights", 
    "yolov3.weights"
)

# Download YOLOv3 config
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", 
    "yolov3.cfg"
)


('yolov3.cfg', <http.client.HTTPMessage at 0x13799e78c10>)

In [15]:
import os
print(os.path.abspath("yolo.cfg"))
print(os.path.abspath("yolo.weights"))


e:\Neko\yolo.cfg
e:\Neko\yolo.weights


In [4]:
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button
from kivy.uix.image import Image
from kivy.uix.filechooser import FileChooserListView
from kivy.graphics.texture import Texture
from PIL import Image as PILImage
import os

In [28]:
# Load YOLO Model
yolo_model = "yolov3.weights"  # Replace with "yolov8.onnx" if using YOLOv8
yolo_cfg = "yolov3.cfg"
coco_labels = "coco.names"  # File containing COCO dataset class labels

# Load class labels
with open(coco_labels, 'r') as f:
    classes = f.read().strip().split("\n")

# Initialize the YOLO model
net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_model)
layer_names = net.getLayerNames()
# Safely get output layers based on OpenCV version
output_indices = net.getUnconnectedOutLayers()
output_layers = [layer_names[i - 1] for i in output_indices.flatten()]


# Load the image
image = cv2.imread('NekoImages\pic.jpg')
(H, W) = image.shape[:2]

# Prepare image for YOLO
blob = cv2.dnn.blobFromImage(image, scalefactor=1 / 255.0, size=(416, 416), swapRB=True, crop=False)
net.setInput(blob)

# Perform object detection
layer_outputs = net.forward(output_layers)

# Process detections
boxes = []
confidences = []
class_ids = []

confidence_threshold = 0.5
nms_threshold = 0.3

for output in layer_outputs:
    for detection in output:
        scores = detection[5:]  # Class scores start from index 5
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > confidence_threshold:
            box = detection[0:4] * np.array([W, H, W, H])
            (center_x, center_y, width, height) = box.astype("int")

            # Calculate top-left corner of the bounding box
            x = int(center_x - (width / 2))
            y = int(center_y - (height / 2))

            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply Non-Maxima Suppression (NMS)
indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

# Draw detected boxes on the image
if len(indices) > 0:
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        
        # Draw bounding box and label
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Display image with bounding boxes
cv2.imshow('Detected Objects', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [30]:
import cv2
import numpy as np

# Assuming `boxes` contains the bounding box coordinates as a result of detection
# boxes format: [(x1, y1, w1, h1), (x2, y2, w2, h2), ...]

# Create a blank mask with the same dimensions as the input image
mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

# Loop through the bounding boxes and fill them on the mask
for (x, y, w, h) in boxes:
    # Draw filled rectangles corresponding to the bounding boxes
    cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)  # White filled rectangle

# Display the generated mask
cv2.imshow('Object Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [34]:
import cv2
import numpy as np

# Assume you already have:
# `image`: Loaded input image
# `mask`: Binary mask of the detected objects (255 for objects, 0 elsewhere)

# Step 1: Copy the image for the text overlay
text_layer = np.zeros_like(image, dtype=np.uint8)  # Create a blank image for the text
image_with_text = image.copy()

# Step 2: Add text to the blank text layer
text = "Text Behind Objects"
text_position = (130, 460)
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Step 3: Invert the mask to make objects opaque
inverted_mask = cv2.bitwise_not(mask)

# Step 4: Use the mask to remove parts of the text behind the objects
text_visible = cv2.bitwise_and(text_layer, text_layer, mask=inverted_mask)

# Step 5: Combine the original image with the visible text
result = cv2.addWeighted(image_with_text, 1, text_visible, 1, 0)

# Step 6: Display the final image
cv2.imshow('Image with Text Behind Objects', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
# !pip install ultralytics
from ultralytics import YOLO
from PIL import Image
model = YOLO("yolov8m-seg.pt")
results = model.predict("NekoImages\pic.jpg")
result = results[0]
masks = result.masks
len(masks)



image 1/1 e:\Neko\NekoImages\pic.jpg: 640x480 3 persons, 1 cell phone, 3188.5ms
Speed: 46.2ms preprocess, 3188.5ms inference, 168.7ms postprocess per image at shape (1, 3, 640, 480)


4

In [9]:
# !pip install ultralytics
from ultralytics import YOLO
from PIL import Image
model = YOLO("yolov8m-seg.pt")
results = model.predict("NekoImages\pic.jpg")
result = results[0]
masks = result.masks
len(masks)
mask1 = masks[0]
mask = mask1.data[0].numpy()
polygon = mask1.xy[0]
mask_img = Image.fromarray(mask,"I")
# mask_img.show()
from PIL import ImageDraw
img = Image.open("NekoImages\pic.jpg")
draw = ImageDraw.Draw(img)
draw.polygon(polygon,outline=(0,255,0), width=5)
img.show()
mask2 = masks[1]
mask = mask2.data[0].numpy()
polygon = mask2.xy[0]
mask_img = Image.fromarray(mask,"I")
# mask_img.show()




image 1/1 e:\Neko\NekoImages\pic.jpg: 640x480 3 persons, 1 cell phone, 950.3ms
Speed: 0.0ms preprocess, 950.3ms inference, 9.9ms postprocess per image at shape (1, 3, 640, 480)


drawing segment on one object

In [10]:
from ultralytics import YOLO
from PIL import Image, ImageDraw

# Load the YOLOv8 segmentation model
model = YOLO("yolov8m-seg.pt")

# Predict the results from the image
results = model.predict("NekoImages\pic.jpg")

# Extract results from the first prediction
result = results[0]
masks = result.masks

# Load the input image
img = Image.open("NekoImages\pic.jpg")
draw = ImageDraw.Draw(img)

# Loop through each detection's mask and draw polygons for each
for mask_idx in range(len(masks)):
    # Get the mask data and polygon coordinates
    mask = masks[mask_idx].data[0].numpy()
    polygon = masks[mask_idx].xy[0]

    # Draw the polygon on the image (we use green for the outline, and width = 5)
    draw.polygon(polygon, outline=(0, 255, 0), width=5)

# Display the image with the polygons
img.show()



image 1/1 e:\Neko\NekoImages\pic.jpg: 640x480 3 persons, 1 cell phone, 1105.4ms
Speed: 7.1ms preprocess, 1105.4ms inference, 10.1ms postprocess per image at shape (1, 3, 640, 480)


drawing polygon on all objects detected

In [12]:
# !pip install onnxruntime
import onnxruntime as ort

model = ort.InferenceSession("yolov8m-seg.onnx")

from PIL import Image
import numpy as np

img = Image.open("NekoImages\pic.jpg")

# save original image size for future
img_width, img_height = img.size;
# convert image to RGB,
img = img.convert("RGB");
# resize to 640x640
img = img.resize((640,640))
img.show()
# convert the image to tensor 
# of [1,3,640,640] as required for 
# the model input
input = np.array(img)
input = input.transpose(2,0,1)
input = input.reshape(1,3,640,640).astype('float32')
input = input/255.0
outputs = model.get_outputs()
len(outputs)


2

In [13]:
# !pip install ultralytics
# !pip install onnxruntime
# !pip install opencv-python
# !pip install numpy
# !pip install pillow
# Export YOLOv8 model to ONNX
# model = YOLO("yolov8m-seg.pt")
# model.export(format="onnx")

from ultralytics import YOLO
import onnxruntime as ort
from PIL import Image, ImageDraw
import numpy as np
import cv2

img = Image.open("NekoImages\pic.jpg")
img_width, img_height = img.size
img = img.convert("RGB")
img = img.resize((640,640))

input = np.array(img)
input = input.transpose(2, 0, 1)
input = input.reshape(1,3,640,640).astype("float32")
input = input/255.0
input.shape

# Run the model
model = ort.InferenceSession("yolov8m-seg.onnx")
outputs = model.run(None, {"images":input})
print(len(outputs))

# Process the outputs

output0 = outputs[0]
output1 = outputs[1]
print("Output0:",output0.shape,"Output1:",output1.shape)

output0 = output0[0].transpose()
output1 = output1[0]
print("Output0:",output0.shape,"Output1:",output1.shape)
boxes = output0[:,0:84]
masks = output0[:,84:]
print("Boxes:",boxes.shape,"Masks:",masks.shape)

output1 = output1.reshape(32,160*160)
print(masks.shape,output1.shape)
# Join matrices by matrix multiplication
masks = masks @ output1
print(masks.shape)
# Connect masks to boxes: move all columns from masks matrix to the right of boxes matrix
boxes = np.hstack([boxes,masks])
print(boxes.shape)

# Parse "boxes" to get list of bounding boxes of detected objects,
# their segmentation masks and bounding polygons

yolo_classes = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse",
    "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie",
    "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove",
    "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut",
    "cake", "chair", "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

def intersection(box1,box2):
    box1_x1,box1_y1,box1_x2,box1_y2 = box1[:4]
    box2_x1,box2_y1,box2_x2,box2_y2 = box2[:4]
    x1 = max(box1_x1,box2_x1)
    y1 = max(box1_y1,box2_y1)
    x2 = min(box1_x2,box2_x2)
    y2 = min(box1_y2,box2_y2)
    return (x2-x1)*(y2-y1) 

def union(box1,box2):
    box1_x1,box1_y1,box1_x2,box1_y2 = box1[:4]
    box2_x1,box2_y1,box2_x2,box2_y2 = box2[:4]
    box1_area = (box1_x2-box1_x1)*(box1_y2-box1_y1)
    box2_area = (box2_x2-box2_x1)*(box2_y2-box2_y1)
    return box1_area + box2_area - intersection(box1,box2)

def iou(box1,box2):
    return intersection(box1,box2)/union(box1,box2)

def sigmoid(z):
    return 1/(1 + np.exp(-z))

# parse segmentation mask
def get_mask(row, box, img_width, img_height):
    # convert mask to image (matrix of pixels)
    mask = row.reshape(160,160)
    mask = sigmoid(mask)
    mask = (mask > 0.5).astype("uint8")*255
    # crop the object defined by "box" from mask
    x1,y1,x2,y2 = box
    mask_x1 = round(x1/img_width*160)
    mask_y1 = round(y1/img_height*160)
    mask_x2 = round(x2/img_width*160)
    mask_y2 = round(y2/img_height*160)
    mask = mask[mask_y1:mask_y2,mask_x1:mask_x2]
    # resize the cropped mask to the size of object
    img_mask = Image.fromarray(mask,"L")
    img_mask = img_mask.resize((round(x2-x1),round(y2-y1)))
    mask = np.array(img_mask)
    return mask

# calculate bounding polygon from mask
def get_polygon(mask):
    contours = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    polygon = [[contour[0][0],contour[0][1]] for contour in contours[0][0]]
    return polygon

# parse and filter all boxes
objects = []
for row in boxes:
    xc,yc,w,h = row[:4]
    x1 = (xc-w/2)/640*img_width
    y1 = (yc-h/2)/640*img_height
    x2 = (xc+w/2)/640*img_width
    y2 = (yc+h/2)/640*img_height
    prob = row[4:84].max()
    if prob < 0.5:
        continue
    class_id = row[4:84].argmax()
    label = yolo_classes[class_id]
    mask = get_mask(row[84:25684], (x1,y1,x2,y2), img_width, img_height)
    polygon = get_polygon(mask)
    objects.append([x1,y1,x2,y2,label,prob,mask,polygon])


# apply non-maximum suppression
objects.sort(key=lambda x: x[5], reverse=True)
result = []
while len(objects)>0:
    result.append(objects[0])
    objects = [object for object in objects if iou(object,objects[0])<0.7]

len(result)

# display bounding polygons of detected objects
# on the image
img = Image.open("NekoImages\pic.jpg")
draw = ImageDraw.Draw(img, "RGBA")
for object in result:
    [x1, y1, x2, y2, label, prob, mask, polygon] = object
    # move polygon from (0,0) to the top left point of detected object
    polygon = [(round(x1+point[0]),round(y1+point[1])) for point in polygon]
    draw.polygon(polygon, fill=(0, 255, 0, 125))

img.show()


2
Output0: (1, 116, 8400) Output1: (1, 32, 160, 160)
Output0: (8400, 116) Output1: (32, 160, 160)
Boxes: (8400, 84) Masks: (8400, 32)
(8400, 32) (32, 25600)
(8400, 25600)
(8400, 25684)


drawing segment on 2 object

In [20]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import cv2

# Load the YOLOv8 segmentation model
model = YOLO("yolov8m-seg.pt")

# Predict the results from the image
results = model.predict("NekoImages\\pic.jpg")

# Extract results from the first prediction
result = results[0]
masks = result.masks

# Load the input image
img_path = "NekoImages\\pic.jpg"
img = Image.open(img_path)
draw = ImageDraw.Draw(img)

# Add text layer
text_layer = np.zeros_like(np.array(img), dtype=np.uint8)  # Create a blank image for the text (1-channel, uint8)
text_position = (130, 460)
text = "Text Behind Objects"

# Convert image to BGR for OpenCV, as the text will be written on BGR image
img_cv2 = np.array(img)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2BGR)

# Step 1: Add text to the blank text layer (single channel)
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255), 4)  # Set text color to white and increase size

# Create a mask for the detected objects
object_mask = np.zeros((img_cv2.shape[0], img_cv2.shape[1]), dtype=np.uint8)

# Step 2: Loop through the mask detections and draw polygons around the detected areas
for mask_idx in range(len(masks)):
    # Get the mask data and polygon coordinates for each detected object
    mask = masks[mask_idx].data[0].numpy()
    polygon = masks[mask_idx].xy[0]

    # Draw the polygon on the image (green outline, width = 5)
    draw.polygon(polygon, outline=(0, 255, 0), width=5)

    # Loop through and create a mask for the detected area
    for point in polygon:
        x, y = point
        x, y = int(x), int(y)  # Convert coordinates to integers to avoid IndexError
        if 0 <= x < img_cv2.shape[1] and 0 <= y < img_cv2.shape[0]:
            object_mask[y, x] = 255  # Set pixels within object to 255 for visibility

# Invert the object mask
inverted_mask = cv2.bitwise_not(object_mask)

# Step 3: Use the inverted mask to overlay the text layer on the original image
# The text layer must match the shape of the image (just like the mask)
text_layer_single_channel = cv2.cvtColor(text_layer, cv2.COLOR_BGR2GRAY)  # Convert to 1-channel grayscale if needed

# Ensure the mask and text layer have the same dimensions
inverted_mask = cv2.resize(inverted_mask, (text_layer_single_channel.shape[1], text_layer_single_channel.shape[0]))

# Apply the inverted mask to the text layer
text_visible = cv2.bitwise_and(text_layer_single_channel, text_layer_single_channel, mask=inverted_mask)

# Convert the text_visible to 3 channels to match img_cv2
text_visible = cv2.cvtColor(text_visible, cv2.COLOR_GRAY2BGR)

# Final step: Combine the image with text behind objects and show result
result_image = cv2.addWeighted(img_cv2, 1, text_visible, 1, 0)

# Convert result image back to PIL for display
result_image_pil = Image.fromarray(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))

# Display the final image with text behind objects and polygons
result_image_pil.show()



image 1/1 e:\Neko\NekoImages\pic.jpg: 640x480 3 persons, 1 cell phone, 950.2ms
Speed: 10.1ms preprocess, 950.2ms inference, 23.6ms postprocess per image at shape (1, 3, 640, 480)


In [7]:
from ultralytics import YOLO
from PIL import Image, ImageDraw
import numpy as np
import cv2

# Load the YOLOv8 segmentation model
model = YOLO("yolov8m-seg.pt")

# Predict the results from the image
results = model.predict("NekoImages\\pic.jpg")

# Extract results from the first prediction
result = results[0]
masks = result.masks

# Load the input image
img_path = "NekoImages\\pic.jpg"
img = Image.open(img_path)
img_cv2 = np.array(img)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2BGR)

# Step 1: Create masks for the segmented objects
object_mask = np.zeros((img_cv2.shape[0], img_cv2.shape[1]), dtype=np.uint8)

for mask_idx in range(len(masks)):
    mask = masks[mask_idx].data[0].numpy()
    polygon = masks[mask_idx].xy[0]

    for point in polygon:
        x, y = point
        x, y = int(x), int(y)
        if 0 <= x < img_cv2.shape[1] and 0 <= y < img_cv2.shape[0]:
            object_mask[y, x] = 255

# Display image with bounding boxes
cv2.imshow('Detected Objects', img_cv2)
cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 e:\Neko\NekoImages\pic.jpg: 640x480 3 persons, 1 cell phone, 928.5ms
Speed: 7.0ms preprocess, 928.5ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 480)


In [9]:
# Load the input image
img = Image.open("NekoImages\pic.jpg")
draw = ImageDraw.Draw(img)

# Loop through each detection's mask and draw polygons for each
for mask_idx in range(len(masks)):
    # Get the mask data and polygon coordinates
    mask = masks[mask_idx].data[0].numpy()
    polygon = masks[mask_idx].xy[0]

    # Draw the polygon on the image (we use green for the outline, and width = 5)
    draw.polygon(polygon, outline=(0, 255, 0), width=5)

In [10]:
import cv2
import numpy as np

# Assuming `boxes` contains the bounding box coordinates as a result of detection
# boxes format: [(x1, y1, w1, h1), (x2, y2, w2, h2), ...]

# Create a blank mask with the same dimensions as the input image
# mask = np.zeros((img_cv2.shape[0], img_cv2.shape[1]), dtype=np.uint8)

# # Loop through the bounding boxes and fill them on the mask
# for (x, y, w, h) in boxes:
#     # Draw filled rectangles corresponding to the bounding boxes
#     cv2.rectangle(mask, (x, y), (x + w, y + h), 255, -1)  # White filled rectangle

# Display the generated mask
cv2.imshow('Object Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [34]:
import cv2
import numpy as np

# Assume you already have:
# `image`: Loaded input image
# `mask`: Binary mask of the detected objects (255 for objects, 0 elsewhere)

# Step 1: Copy the image for the text overlay
text_layer = np.zeros_like(img_cv2, dtype=np.uint8)  # Create a blank image for the text
image_with_text = image.copy()

# Step 2: Add text to the blank text layer
text = "Text Behind Objects"
text_position = (130, 460)
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Step 3: Invert the mask to make objects opaque
inverted_mask = cv2.bitwise_not(mask)

# Step 4: Use the mask to remove parts of the text behind the objects
text_visible = cv2.bitwise_and(text_layer, text_layer, mask=inverted_mask)

# Step 5: Combine the original image with the visible text
result = cv2.addWeighted(image_with_text, 1, text_visible, 1, 0)

# Step 6: Display the final image
cv2.imshow('Image with Text Behind Objects', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [36]:
# Step 2: Create a blank image for the text, drawing the text layer in white
text_layer = np.zeros_like(img_cv2, dtype=np.uint8)
text_position = (130, 460)
text = "Text Behind Objects"
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)

# Step 3: Invert the object mask to make the objects opaque and background transparent
inverted_mask = cv2.bitwise_not(object_mask)

# Step 4: Use the inverted mask to show the background (text appears where there are no objects)
background = cv2.bitwise_and(img_cv2, img_cv2, mask=inverted_mask)  # Background (image) without objects

# Step 5: Combine the background with the text where the mask is transparent
result_with_text = cv2.add(background, text_layer)

# Step 6: Now apply the original object mask on top of this image
masked_image = cv2.bitwise_and(img_cv2, img_cv2, mask=object_mask)  # Extract only the objects

# Step 7: Combine the masked image (objects) with the final image (text behind objects)
final_result = cv2.add(result_with_text, masked_image)

# Convert result image back to PIL for display
result_image_pil = Image.fromarray(cv2.cvtColor(final_result, cv2.COLOR_BGR2RGB))

# Display the final image with text behind objects and polygons
result_image_pil.show()

steps wise working code

In [2]:
from ultralytics import YOLO
import numpy as np
import cv2
from PIL import Image, ImageDraw

# Step 1: Load Image and YOLOv8 Model for Segmentation
model = YOLO("yolov8m-seg.pt")
img_path = "neko_WP_20180829_22_40_52_Pro.jpg" #"NekoImages\\pic.jpg"

# Predict the results from the image
results = model.predict(img_path)

# Extract results from the first prediction
result = results[0]
masks = result.masks

# Load the image into OpenCV format
img = Image.open(img_path)
img_cv2 = np.array(img)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2BGR)

# Step 3: Create Masked Layer for Segmented Areas
object_mask = np.zeros((img_cv2.shape[0], img_cv2.shape[1]), dtype=np.uint8)

for mask_idx in range(len(masks)):
    mask = masks[mask_idx].data[0].numpy()
    mask = (mask * 255).astype(np.uint8)  # Convert to binary mask (0 or 255)

    # Resize the mask to match the size of the original image
    mask_resized = cv2.resize(mask, (img_cv2.shape[1], img_cv2.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    # Add this mask to the overall object mask
    object_mask = cv2.bitwise_or(object_mask, mask_resized)

# Step 4: Add Text to Original Image and Mask Where Segments Appear
# Text Overlay on the original image
text_layer = np.zeros_like(img_cv2, dtype=np.uint8)
text_position = (100, 500)  # Text position
text = "Family"
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 10)

# Create an inverted mask of the segmented regions to hide text
inverted_mask = cv2.bitwise_not(object_mask)

# Combine text layer with the original image
text_visible = cv2.addWeighted(img_cv2, 1, text_layer, 1, 0)

# Step 5: Combine Segmented Layer with Text Hidden Under Segments
# Apply the mask to ensure text is hidden behind segmented objects
masked_image = cv2.bitwise_and(img_cv2, img_cv2, mask=object_mask)
background = cv2.bitwise_and(text_visible, text_visible, mask=inverted_mask)
final_image = cv2.add(masked_image, background)

# Convert final image to PIL format for displaying
final_image_pil = Image.fromarray(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))

# Display final result
final_image_pil.show()



image 1/1 e:\Neko\neko_WP_20180829_22_40_52_Pro.jpg: 640x384 2 persons, 1 couch, 1 potted plant, 1 vase, 997.4ms
Speed: 0.0ms preprocess, 997.4ms inference, 22.5ms postprocess per image at shape (1, 3, 640, 384)


In [3]:
from ultralytics import YOLO
import numpy as np
import cv2
from PIL import Image

# Step 1: Load Image and YOLOv8 Model for Segmentation
model = YOLO("yolov8m-seg.pt")
img_path = "neko_WP_20180829_22_40_52_Pro.jpg"  # or your image path

# Predict the results from the image
results = model.predict(img_path)

# Extract results from the first prediction
result = results[0]
masks = result.masks
class_labels = result.names  # Get class labels
classes_to_keep = ["person", "dog", "cat", "horse", "sheep", "cow"]  # Define classes to keep

# Load the image into OpenCV format
img = Image.open(img_path)
img_cv2 = np.array(img)
img_cv2 = cv2.cvtColor(img_cv2, cv2.COLOR_RGB2BGR)

# Step 3: Create Masked Layer for Segmented Areas
object_mask = np.zeros((img_cv2.shape[0], img_cv2.shape[1]), dtype=np.uint8)

# Iterate through the results and apply mask only for the selected classes
for mask_idx in range(len(masks)):
    class_idx = int(result.boxes.cls[mask_idx].item())  # Get class index for current mask
    class_name = class_labels[class_idx]  # Get the class name

    # Only process masks for person and animal classes
    if class_name in classes_to_keep:
        mask = masks[mask_idx].data[0].numpy()
        mask = (mask * 255).astype(np.uint8)  # Convert to binary mask (0 or 255)

        # Resize the mask to match the size of the original image
        mask_resized = cv2.resize(mask, (img_cv2.shape[1], img_cv2.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        # Add this mask to the overall object mask
        object_mask = cv2.bitwise_or(object_mask, mask_resized)

# Step 4: Add Text to Original Image and Mask Where Segments Appear
# Text Overlay on the original image
text_layer = np.zeros_like(img_cv2, dtype=np.uint8)
text_position = (100, 500)  # Text position
text = "Family"
cv2.putText(text_layer, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 10)

# Create an inverted mask of the segmented regions to hide text
inverted_mask = cv2.bitwise_not(object_mask)

# Combine text layer with the original image
text_visible = cv2.addWeighted(img_cv2, 1, text_layer, 1, 0)

# Step 5: Combine Segmented Layer with Text Hidden Under Segments
# Apply the mask to ensure text is hidden behind segmented objects
masked_image = cv2.bitwise_and(img_cv2, img_cv2, mask=object_mask)
background = cv2.bitwise_and(text_visible, text_visible, mask=inverted_mask)
final_image = cv2.add(masked_image, background)

# Convert final image to PIL format for displaying
final_image_pil = Image.fromarray(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))

# Display final result
final_image_pil.show()



image 1/1 e:\Neko\neko_WP_20180829_22_40_52_Pro.jpg: 640x384 2 persons, 1 couch, 1 potted plant, 1 vase, 931.5ms
Speed: 0.0ms preprocess, 931.5ms inference, 11.4ms postprocess per image at shape (1, 3, 640, 384)


In [8]:
def beautify_segment(image, mask):
        if image is None or mask is None:
            return image  # Return the original image if no mask or image is provided

        # Ensure the mask is resized to match the image dimensions
        if mask.shape[:2] != image.shape[:2]:
            mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)

        # Convert mask to a single channel if needed
        if len(mask.shape) > 2:
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        
        # Convert mask to 8-bit unsigned integer type
        mask = mask.astype(np.uint8)

        # Debug prints
        print("Image shape:", image.shape)
        print("Mask shape (before resize):", mask.shape)

        # Extract the masked area
        masked_area = cv2.bitwise_and(image, image, mask=mask)
        
        # Apply bilateral filtering for edge-preserving smoothing
        smoothed = cv2.bilateralFilter(masked_area, 5, 85, 85)
        
        # Slightly enhance contrast and brightness of the masked area (subtle beautification)
        enhanced = cv2.convertScaleAbs(smoothed, alpha=1.1, beta=10)
        
        # Create a blurred mask for smooth edges
        blurred_mask = cv2.GaussianBlur(mask, (5, 5), 1)
        blurred_mask = blurred_mask.astype(np.float32) / 255.0

        # Ensure blurred_mask has the same number of channels as enhanced
        if len(blurred_mask.shape) == 2:
            blurred_mask = cv2.cvtColor(blurred_mask, cv2.COLOR_GRAY2BGR)

        # Debug prints
        print("Mask shape (after resize):", blurred_mask.shape)
        
        # Blend the enhanced region with the original image using the blurred mask
        masked_correction = cv2.multiply(enhanced.astype(np.float32), blurred_mask)
        original_contribution = cv2.multiply(image.astype(np.float32), 1 - blurred_mask)
        blended_masked_area = cv2.add(masked_correction, original_contribution).astype(np.uint8)
        
        # Merge the processed masked region back to the original image
        mask_inverse = cv2.bitwise_not(mask)
        untouched_area = cv2.bitwise_and(image, image, mask=mask_inverse)
        beautified_image = cv2.add(untouched_area, blended_masked_area)
        
        # Optionally enhance the entire image slightly
        beautified_image = cv2.convertScaleAbs(beautified_image, alpha=1.05, beta=5)
        
        return beautified_image

img_cv2 = beautify_segment(img_cv2, object_mask)

# Convert final image to PIL format for displaying
final_image_pil = Image.fromarray(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))

# Display final result
final_image_pil.show()

Image shape: (1918, 1077, 3)
Mask shape (before resize): (1918, 1077)
Mask shape (after resize): (1918, 1077, 3)


In [9]:
!sudo apt update
!sudo apt install -y python3-pip openjdk-11-jdk git unzip zlib1g-dev lib32stdc++6 lib32z1


'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
